In [1]:
import pickle

In [2]:
x_path = 'C:/Users/Carrt/OneDrive/Code/Motion Privacy/External Repositories/Skeleton-anonymization/X_unet.pkl'

with open(x_path, 'rb') as f:
    X = pickle.load(f)

In [11]:
def anonymizer_to_rf(t):
    # (75)
    # [x:0,y:1,z:2][300][joints][actors]
    xyz, frames, joints, actors = t.shape
    # transpose to make loop simple
    # [frame][actors][joints][xyz]
    t = t.transpose(1, 3, 2, 0)
    # make empty array
    frames = []

    # crazy loop
    for frame in t:
        f = []
        for actor in frame:
            for joint in actor:
                for xyz in joint:
                    f.append(xyz)
      
        # Cut to 75
        if len(f) != 75:
            f = f[:75]

        frames.append(f)

    return frames

In [17]:
temp = anonymizer_to_rf(X['P001'][0])[0]

In [23]:
import bpy

# Create armature object
arm = bpy.data.armatures.new('Skeleton')
arm_ob = bpy.data.objects.new('Skeleton', arm)
bpy.context.scene.collection.objects.link(arm_ob)

# Set edit mode to add bones
bpy.ops.object.mode_set(mode='EDIT')

# Define bone hierarchy and names
bones = {
    'Hips': ['Spine1'],
    'Spine1': ['Spine2'],
    'Spine2': ['Neck'],
    'Neck': ['Head'],
    'Spine1': ['ShoulderLeft', 'ShoulderRight'],
    'ShoulderLeft': ['ElbowLeft'],
    'ElbowLeft': ['WristLeft'],
    'WristLeft': ['HandLeft'],
    'ShoulderRight': ['ElbowRight'],
    'ElbowRight': ['WristRight'],
    'WristRight': ['HandRight'],
    'Hips': ['HipLeft', 'HipRight'],
    'HipLeft': ['KneeLeft'],
    'KneeLeft': ['AnkleLeft'],
    'AnkleLeft': ['FootLeft'],
    'HipRight': ['KneeRight'],
    'KneeRight': ['AnkleRight'],
    'AnkleRight': ['FootRight'],
}

bone_names = list(bones.keys())

# Add root bone
hip_bone = arm.edit_bones.new('Hips')
hip_bone.head = (0, 0, 0)
hip_bone.tail = (0, 0, 1)

# Add remaining bones and position them
for bone_name in bone_names:
    bone = arm.edit_bones.new(bone_name)
    if bone_name == 'Hips':
        continue
    parent_bone_name = next(
        k for k, v in bones.items() if bone_name in v
    )
    bone.parent = arm.edit_bones[parent_bone_name]
    bone.head = bone.parent.tail
    bone.tail = bone.parent.tail + (0, 0, 1)

# Set back to object mode
bpy.ops.object.mode_set(mode='OBJECT')


ModuleNotFoundError: No module named 'bpy'